In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Polygon
import math
from Visual import MapViser
from utils import calcu_lcoe
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import numpy as np

In [77]:
def get_crf(wacc, years):
    crf = (wacc * math.pow(1+wacc, years)) / (math.pow(1+wacc, years)-1)

    return round(crf,6)

In [120]:
valid_cell = gpd.read_file('./data/res/valid_cell.gpkg')

In [145]:
scen = 'PowerOpex2.0X'

In [146]:
aidc_power = 30 #MW

capex = 823690000

fom = 0.0065 * capex

ele_cost_base = {
    'China':0.72, #yuan/kWh
    'Africa':1.4,
    'North America':1.68,
    'South America':1.155,
    'Europe':1.4,
    'Oceania':1.2,
    'India':0.91,
    'Asia':1.0 #Asia outside China
}

pue_base = 1.2
wacc_base = 0.075
building_yr = 20

fom_desert = fom * 1.5 * 1
pv_opex_ratio = 0.005 * 2
we_opex_ratio = 0.015 * 2
st_opex_ratio = 0.005 * 2
ope_years = 25

sto_dep_coef = 1.0 * 1.0

sto_capex_coef = 1.0 * 1

cr_max = 4.01
pr_max = 4.01

In [147]:
capex_config = {'China':{'pv':2200,'we':3000,'st':1500},
                'Africa':{'pv':5300*2200/3450,'we':8400*3000/4200,'st':8008*1500/2400},
                'North America':{'pv':7000*2200/3450,'we':11900*3000/4200,'st':8008*1500/2400},
                'South America':{'pv':5200*2200/3450,'we':5850*3000/4200,'st':9500*1500/2400},
                'Asia':{'pv':4081*2200/3450,'we':7800*3000/4200,'st':7000*1500/2400},
                'Oceania':{'pv':5155*2200/3450,'we':10655*3000/4200,'st':11130*1500/2400},
                'Europe':{'pv':7560*2200/3450,'we':10920*3000/4200,'st':8400*1500/2400},
                'India':{'pv':3400*2200/3450,'we':5882*3000/4200,'st':3700*1500/2400}}

In [148]:
for k in capex_config:
    capex_config[k]['st'] = sto_capex_coef * capex_config[k]['st']

In [149]:

valid_cell['lcoe_sens'] = -1.0

for i,r in valid_cell.iterrows():
    capex_tag = r['capex_tag']
    
    power_opex = (capex_config[capex_tag]['pv'] * pv_opex_ratio + 
                  capex_config[capex_tag]['we'] * we_opex_ratio + 
                  capex_config[capex_tag]['st'] * st_opex_ratio)
    
    power_capex = (capex_config[capex_tag]['pv'] * r['pv_ratio'] + 
                   capex_config[capex_tag]['we'] * r['we_ratio'] + 
                   capex_config[capex_tag]['st'] * r['st_ratio'] * sto_dep_coef)
    
    lcoe = calcu_lcoe(capex=power_capex,opex=power_opex,cf=1,years=ope_years,disc_r=0.074)
    
    valid_cell.at[i,'lcoe_sens'] = lcoe
        

In [150]:
valid_cell['ele_price_base'] = -1.0

for k in ele_cost_base:
    valid_cell.loc[valid_cell['capex_tag']==k,'ele_price_base'] = ele_cost_base[k]
    
valid_cell['ele_cost_base'] = (valid_cell['ele_price_base'] *
                               aidc_power * 
                               pue_base *
                               0.75 * 
                               8760 * 
                               1000)


In [151]:
profit_scens = {'CapEx Rate':[],
                'PUE Rate':[],
                'WACC Rate':[],
                'Ele Potential':[],
                'Cap Potential':[]}

In [152]:
valid_cell['elecost_savings'] = -1.0

In [153]:
cr_range = np.arange(1,cr_max,0.01)
pr_range = np.arange(1,pr_max,0.01)
wr_range = np.arange(2,3)


for cr in cr_range:
    for pr in pr_range:
        for wr in wr_range:
            elecons_sens = aidc_power * pue_base * pr * 0.75 * 8760 * 1000
            
            exp_base = capex * get_crf(wacc=wacc_base*wr,years=building_yr) + fom
            exp_gap = capex * cr * get_crf(wacc=wacc_base*wr,years=building_yr) + fom_desert - exp_base
            
            valid_cell['elecost_savings'] = valid_cell['ele_cost_base'] - elecons_sens * valid_cell['lcoe_sens']
            
            cap_count = valid_cell.loc[valid_cell['elecost_savings']>exp_gap]['equal_cap'].sum()
            ele_count = cap_count  * 8760 * 1e-6
            
            profit_scens['CapEx Rate'].append(cr)
            profit_scens['PUE Rate'].append(pr)
            profit_scens['WACC Rate'].append(wr)
            profit_scens['Ele Potential'].append(ele_count)
            profit_scens['Cap Potential'].append(cap_count)

In [154]:
profit_scens = pd.DataFrame.from_dict(profit_scens)

In [ ]:
import numpy as np

plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 30

cmap = 'RdPu'
#cmap = 'BuGn'

fig,ax = plt.subplots(figsize=(16,14),dpi=400)

vis = profit_scens

xi = np.arange(1,cr_max,0.01)
yi = np.arange(1,pr_max,0.01)

X, Y = np.meshgrid(xi, yi)

Z = np.array(vis['Ele Potential']).reshape(len(xi),len(yi))

contour = ax.contourf(X, Y, Z, 25, cmap=cmap)

ax.set_title('WACC: 15.0%')

# 同时，我们可以叠加绘制等高线
CS = ax.contour(X, Y, Z, 25, colors='grey', linewidths=0.75)

# 6. 添加文字标记 (clabel)
label_levels = np.arange(10, 240, 40)

plt.colorbar(contour, label='Electricity potential (PWh/yr)')

ax.set_xlabel('CapEx (Non-IT) ratio to the real case')

ax.set_ylabel('PUE ratio to the real case')

ax.set_xticks(np.arange(1.0,cr_max,0.5),
                [str(x)+'X' for x in np.arange(1.0,cr_max,0.5)])

ax.set_yticks(np.arange(1.0,pr_max,0.5),
                [str(x)+'X' for x in np.arange(1.0,pr_max,0.5)])

ax.minorticks_on()
                
ax.tick_params(axis='y', 
                which='major', 
                length=8, 
                color='black')

ax.tick_params(axis='y', 
                which='minor', 
                length=4, 
                color='black')

ax.tick_params(axis='x', 
                which='major', 
                length=8, 
                color='black')

ax.tick_params(axis='x', 
                which='minor', 
                length=4, 
                color='black')
        
plt.savefig('./data/fig/Vis/EcoSens/profit_poten_'+scen+'.jpg',bbox_inches='tight')
plt.savefig('./data/fig/Vis/EcoSens/profit_poten_'+scen+'.pdf',bbox_inches='tight')

plt.close()

In [156]:
profit_scens.to_pickle('./data/res/EcoSens/'+scen+'.pkl')